---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    #u_town = pd.read_
    #test_df = pd.DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    #columns=["State", "RegionName"]  )
    
    ## open university_towns.txt
    ut_file = open('university_towns.txt', 'r')
    ut_text = ut_file.readlines()
    states_towns = []
    for line in ut_text:
        if "[edit]" in line:
            state_name = line.split("[")[0]
        else:
            states_towns.append([state_name, line.split("(")[0].strip()])
    
    ut_df = pd.DataFrame( states_towns, columns=["State", "RegionName"]  )
    return ut_df


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # skip rows until we get to 2000q1 - orginally was skiprows=7 to get to quarter values
    gdp = pd.read_excel('gdplev.xls', skiprows=219, index_col=None, names=None, parse_cols=[4,6], parse_dates=False)
    gdp.columns = ['Quarter', 'GDP in billions of chained 2009 dollars']
    gdp = gdp.set_index('Quarter')
    #recession_start_qtr = None
    one_qtr_ago = 0
    one_qtr_ago_index = None
    two_qtr_ago = 0
    two_qtr_ago_index = None
    
    
    for index, row in gdp.iterrows():
        #print(row['GDP in billions of chained 2009 dollars'])

        if (row['GDP in billions of chained 2009 dollars'] < one_qtr_ago) and (one_qtr_ago < two_qtr_ago):
            return one_qtr_ago_index

        else:
            two_qtr_ago = one_qtr_ago
            two_qtr_ago_index = one_qtr_ago_index
            one_qtr_ago = row['GDP in billions of chained 2009 dollars']
            one_qtr_ago_index = index


    return None



In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_start = get_recession_start()
    
    gdp = pd.read_excel('gdplev.xls', skiprows=219, index_col=None, names=None, parse_cols=[4,6], parse_dates=False)
    gdp.columns = ['Quarter', 'GDP in billions of chained 2009 dollars']
    gdp = gdp.set_index('Quarter')
    one_qtr_ago = 0
    one_qtr_ago_index = None
    two_qtr_ago = 0
    two_qtr_ago_index = None
    
    # get index location of recession_start
    rs_loc = gdp.index.get_loc(recession_start)
    #print(rs_loc)
    # drop records before recession_start 
    gdp = gdp.drop(gdp.index[0:rs_loc])
    #print(gdp)
       
    for index, row in gdp.iterrows():
        #print(row['GDP in billions of chained 2009 dollars'])

        if (row['GDP in billions of chained 2009 dollars'] > one_qtr_ago) and (one_qtr_ago > two_qtr_ago):
            return index

        else:
            two_qtr_ago = one_qtr_ago
            two_qtr_ago_index = one_qtr_ago_index
            one_qtr_ago = row['GDP in billions of chained 2009 dollars']
            one_qtr_ago_index = index


    return None



In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows=219, index_col=None, names=None, parse_cols=[4,6], parse_dates=False)
    gdp.columns = ['Quarter', 'GDP in billions of chained 2009 dollars']
    gdp = gdp.set_index('Quarter')
    
    # get index values of recession_start and recession_end and drop rows that are outside of this 
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    
    rs_loc = gdp.index.get_loc(recession_start)
    re_loc = gdp.index.get_loc(recession_end)
    
    gdp = gdp.drop(gdp.index[re_loc + 1:])
    gdp = gdp.drop(gdp.index[0:rs_loc])
    
    
    
    return gdp['GDP in billions of chained 2009 dollars'].idxmin()


In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    #print(housing)
    housing = housing.drop(housing.columns[range(6,51)], axis=1)
    #print(housing)
    
    # build list of quarter beginnings 
    months = housing.columns.values
    months = months[6:] # remove non-monthly periods
    quarter_begin = months[0::3] # use numpy slicing 
    
    # build list of column names with regular list
    column_names = []
    for month in quarter_begin:
        year = month.split('-')[0]
        if month.split('-')[1] == '01':
            quarter = 'q1'
        elif month.split('-')[1] == '04':
            quarter = 'q2'
        elif month.split('-')[1] == '07':
            quarter = 'q3'
        else:
            quarter = 'q4'
        column_names.append(year + quarter)
        
    # create series for column_names
    columns = pd.Series(column_names, index=quarter_begin)
    
    # build new df and begin populating  
    target_housing = housing.copy(deep=True)
    target_housing = target_housing.drop(target_housing.columns[6:], axis=1)
    
    for start_month, quarter in columns.iteritems():
        pos = housing.columns.get_loc(start_month)
        data = housing.iloc[:, pos:pos+3]
        target_housing[quarter] = data.mean(axis=1)
    # .loc['2001-01']
    
    # change 'State' from abbreviation to full name, and set index 
    target_housing['State'] = [states[state] for state in target_housing['State']]
    target_housing.set_index(["State", "RegionName"], inplace =True)
    
    # remove unneeded data
    target_housing = target_housing.drop(target_housing.columns[0:4], axis=1)
    
    return target_housing
    #return housing

#convert_housing_data_to_quarters()
#convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    hdf = convert_housing_data_to_quarters()
    ul = get_list_of_university_towns()
    r_start = get_recession_start()
    #print(r_start)
    r_bottom = get_recession_bottom()
    #print(hdf.columns.get_loc(r_start))
    
    # remove data after r_bottom from hdf
    #hdf = hdf.drop(hdf.columns[hdf.columns.get_loc(r_bottom)+1:], axis=1)
    #hdf = hdf.drop(hdf.columns[0 : hdf.columns.get_loc(r_start)], axis=1)
    hdf['PriceRatio'] = hdf.iloc[:, hdf.columns.get_loc(r_start) - 1].divide(hdf['2009q2']) # calculate pricing difference
    hdf = hdf.reset_index()    
    
    # create university towns df
    #udf = hdf.merge(ul, how="inner", on=["State", "RegionName"])#left_on=["State", "RegionName"], right_on=["State", "RegionName"])
    udf = pd.merge(ul, hdf, how="inner", left_on=["RegionName", "State"], right_on=["RegionName", "State"])
    #udf = pd.merge(ul, hdf, how="left", left_on=["RegionName", "State"], right_on=["RegionName", "State"])
    # create non-university towns df
    keys = ['State', 'RegionName']
    i1 = hdf.set_index(keys).index
    i2 = udf.set_index(keys).index
    ndf = hdf[~i1.isin(i2)]
    ndf = ndf.reset_index()
    ndf = ndf.drop(ndf.columns[0], axis=1)
    
    st,p = ttest_ind(udf['PriceRatio'], ndf['PriceRatio'], nan_policy='omit')
    
    if p < 0.01:
        return True, p, "university town"
    
    else:
        return False, p, "university town"
    

run_ttest()

(True, 0.0027240637047531249, 'university town')